<a href="https://colab.research.google.com/github/villasen/colab_notebooks/blob/master/Keras_train_for_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/villasen/speech-numpy.git

In [0]:
!git clone https://github.com/villasen/Speech-tiny.git

In [6]:
import keras
from keras import layers
from keras import models
from keras import backend as K
from keras import optimizers
from keras import utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import numpy as np
import librosa
import tensorflow
import scipy
import os, shutil
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from six.moves import urllib
import sys
import tarfile

Using TensorFlow backend.


In [0]:
dest_directory = "/content/speech_target/"
data_url='http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
DATA_PATH= '/content/Speech-tiny/'

In [0]:
!ls -al speech-numpy/

In [0]:
!ls -al speech_target

In [0]:
!rm -r sample_data

In [0]:
#!rm -r speech-numpy/README.md
#!rm -r speech-numpy/.ipynb_checkpoints/
!rm -r speech-numpy/.git/

In [0]:

data_url='http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
os.makedirs(dest_directory)
filename = data_url.split('/')[-1]
filepath = os.path.join(dest_directory, filename)

print(filename)
print(filepath)
 
def _progress(count, block_size, total_size):
    sys.stdout.write('\r>> Downloading %s %.1f%%' % \
            (filename, float(count * block_size) / float(total_size) * 100.0)) 
    sys.stdout.flush()

urban_dataset_dir = '/content/combined_datasets/' 

In [0]:
!rm -r /content/target_npy_files/

In [0]:

  
def maybe_download_and_extract_dataset(data_url, dest_directory):
    """Download and extract data set tar file.
    If the data set we're using doesn't already exist, this function
    downloads it from the TensorFlow.org website and unpacks it into a
    directory.
    If the data_url is none, don't download anything and expect the data
    directory to contain the correct files already.
    Args:
      data_url: Web location of the tar file containing the data set.
      dest_directory: File path to extract data to.
    """
    if not data_url:
      return
    if not os.path.exists(dest_directory):
      os.makedirs(dest_directory)
    filename = data_url.split('/')[-1]
    filepath = os.path.join(dest_directory, filename)
    print(filepath)
    if not os.path.exists(filepath):

      def _progress(count, block_size, total_size):
        sys.stdout.write(
            '\r>> Downloading %s %.1f%%' %
            (filename, float(count * block_size) / float(total_size) * 100.0))
        sys.stdout.flush()

      try:
        filepath, _ = urllib.request.urlretrieve(data_url, filepath, _progress)
      except:
        tf.logging.error('Failed to download URL: %s to folder: %s', data_url,
                         filepath)
        tf.logging.error('Please make sure you have enough free space and'
                         ' an internet connection')
        raise
      print()
      statinfo = os.stat(filepath)
      tf.logging.info('Successfully downloaded %s (%d bytes)', filename,
                      statinfo.st_size)
    tarfile.open(filepath, 'r:gz').extractall(dest_directory)

In [8]:
maybe_download_and_extract_dataset( data_url, dest_directory)

/content/speech_target/speech_commands_v0.02.tar.gz
>> Downloading speech_commands_v0.02.tar.gz 100.0%
INFO:tensorflow:Successfully downloaded speech_commands_v0.02.tar.gz (2428923189 bytes)


In [0]:
!ls speech_target/ -al

In [0]:
labels = os.listdir('speech-numpy/')
print(labels)

In [49]:
!rm -r /content/test_single_file/

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [0]:
#test_single_file = '/content/test_single_file/'
test_single_file = '/content/test_single_file/'
os.mkdir(test_single_file)




#!mv speech_target/bed/00176480_nohash_0.wav test_single_file

In [0]:
os.chdir('/content/')

In [0]:
os.mkdir('/content/test_single_file/cat')

In [0]:
!mv /content/Speech-tiny/

In [0]:
#os.chdir(test_single_file)
#os.mkdir('bed')
!mv /content/Speech-tiny/learn/234d6a48_nohash_1.wav /content/test_single_file/learn/

In [0]:

target_dir = '/content/target_npy_files'
target_model = '/content/target_model/'
#DATA_PATH = "small-urban-sound-dataset/tiny-dataset/"
#DATA_PATH = "small-urban-sound-dataset/combined_datasets/"
DATA_PATH= '/content/Speech-tiny/'
os.mkdir(target_dir)




def wav2mfcc(file_path, max_pad_len):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    mfcc = librosa.feature.mfcc(wave, sr=16000, n_mfcc=10, n_fft=640, hop_length=320)
    pad_width = max_pad_len - mfcc.shape[1]
    if pad_width < 0: 
      print(mfcc.shape[1])
      print(pad_width)
      print("error in "+ file_path)
    
    mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfcc  

  
  
def get_labels(path):
    labels = os.listdir(path) 
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)  

  
def save_data_to_array(path=DATA_PATH, max_pad_len=51):
    labels, _, _ = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in wavfiles:
            if label == '_background_noise_' : break
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len)
            
            mfcc_vectors.append(mfcc)
        np.save('/content/target_npy_files/' + label + '.npy', mfcc_vectors)
        print(label)
        print(len(mfcc_vectors))
          

def get_train_test(split_ratio=0.9, random_state=42):
    # Get available labels
    labels, indices, _ = get_labels(DATA_PATH)
    print(labels)
    # Getting first arrays
    X = np.load('/content/target_npy_files/' + labels[0] + '.npy')
    #X = np.load('/content/speech-numpy/' + labels[0])
    y = np.zeros(X.shape[0])

    
    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load('/content/target_npy_files/' + label + '.npy')
        #x = np.load('/content/speech-numpy/' + label)
        
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))
        
    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)

In [0]:

#target_dir = '/content/target_npy_files'
target_dir = /content/target_unknown_npy
target_model = '/content/target_model/'
#DATA_PATH = "small-urban-sound-dataset/tiny-dataset/"
#DATA_PATH = "small-urban-sound-dataset/combined_datasets/"
DATA_PATH= '/content/speech_target/'
os.mkdir(target_dir)




def wav2mfcc(file_path, max_pad_len):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    mfcc = librosa.feature.mfcc(wave, sr=16000, n_mfcc=10, n_fft=640, hop_length=320)
    pad_width = max_pad_len - mfcc.shape[1]
    if pad_width < 0: 
      print(mfcc.shape[1])
      print(pad_width)
      print("error in "+ file_path)
    
    mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfcc  

  
  
def get_labels(path):
    labels = os.listdir(path) 
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)  

  
def save_data_to_array(path=DATA_PATH, max_pad_len=51):
    labels, _, _ = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in wavfiles:
            if label == '_background_noise_' : break
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len)
            
            mfcc_vectors.append(mfcc)
        np.save('/content/target_npy_files/' + label + '.npy', mfcc_vectors)
        print(label)
        print(len(mfcc_vectors))

        
        
def save_data_to_unknown_array(path=DATA_PATH, max_pad_len=51):
  labels, _, _ = get_labels(path)
  
  for label in labels:
    mfcc_vectors = []
    
    wavefile = [path + label + '/' + wavfile for wavefile in os.listdir(path + '/' + label)]
    

def get_train_test(split_ratio=0.9, random_state=42):
    # Get available labels
    labels, indices, _ = get_labels(DATA_PATH)
    print(labels)
    # Getting first arrays
    X = np.load('/content/target_npy_files/' + labels[0] + '.npy')
    #X = np.load('/content/speech-numpy/' + labels[0])
    y = np.zeros(X.shape[0])

    
    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load('/content/target_npy_files/' + label + '.npy')
        #x = np.load('/content/speech-numpy/' + label)
        
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))
        
    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)

In [0]:
os.listdir('Speech-tiny/')

In [0]:
labels, indices, _ = get_labels('Speech-tiny/')

In [0]:
print(labels)

In [0]:
save_data_to_array(path=DATA_PATH, max_pad_len=51)    
X_train, X_test, y_train, y_test = get_train_test()

X_train = X_train.reshape(X_train.shape[0], 10, 51, 1)
X_test = X_test.reshape(X_test.shape[0], 10, 51, 1)
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

In [0]:
!rm -r target_model/


In [0]:
# Creating Keras sequential model
#bn = 1
target_model = '/content/target_model/'
os.mkdir(target_model)
BN=True
model = models.Sequential()

def dscnn_train():
  # 1
      model.add(layers.Conv2D(64, (4,10), strides=(2,2), padding='same', activation='relu', \
                #input_shape=(10, 49, 1)))
                input_shape=(10,51,1)))
      #model.add(layers.Dropout(0.5))
      if BN == True:
          model.add(layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, \
                                                center=True, scale=True, beta_initializer='zeros', \
                                                gamma_initializer='ones', moving_mean_initializer='zeros', \
                                                moving_variance_initializer='ones', beta_regularizer=None, \
                                                gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
      
      #model.add(layers.Dropout(0.5))
      model.add(layers.ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
      #model.add(layers.Dropout(0.5))
      # 2
      model.add(layers.SeparableConv2D(64, (3,3), strides=(1,1), data_format='channels_last', padding='same', depth_multiplier=1, activation='relu'))  
      if BN == True:
          model.add(layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, \
                                              center=True, scale=True, beta_initializer='zeros', \
                                              gamma_initializer='ones', moving_mean_initializer='zeros', \
                                              moving_variance_initializer='ones', beta_regularizer=None, \
                                              gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
      #model.add(layers.Dropout(0.5))
      model.add(layers.ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
      #model.add(layers.Dropout(0.5))

      model.add(layers.Conv2D(64, (1,1), strides=(1,1), padding='same', use_bias=False))
      if BN == True:
          model.add(layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, \
                                              center=True, scale=True, beta_initializer='zeros', \
                                              gamma_initializer='ones', moving_mean_initializer='zeros', \
                                              moving_variance_initializer='ones', beta_regularizer=None, \
                                              gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
     # model.add(layers.Dropout(0.5))
      model.add(layers.ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
     # model.add(layers.Dropout(0.5))
      # 3
      model.add(layers.SeparableConv2D(64, (3,3), strides=(1,1), data_format='channels_last', padding='same', depth_multiplier=1, activation='relu'))
      if BN == True:
          model.add(layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, \
                                              center=True, scale=True, beta_initializer='zeros', \
                                              gamma_initializer='ones', moving_mean_initializer='zeros', \
                                              moving_variance_initializer='ones', beta_regularizer=None, \
                                              gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
      
      #model.add(layers.Dropout(0.5))
      model.add(layers.ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
     # model.add(layers.Dropout(0.5))
      
      model.add(layers.Conv2D(64, (1,1), strides=(1,1), padding='same', use_bias=False))
      if BN == True:
          model.add(layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, \
                                              center=True, scale=True, beta_initializer='zeros', \
                                              gamma_initializer='ones', moving_mean_initializer='zeros', \
                                              moving_variance_initializer='ones', beta_regularizer=None, \
                                              gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
      #model.add(layers.Dropout(0.5))
      model.add(layers.ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
      model.add(layers.Dropout(0.5))

      # 4
      model.add(layers.SeparableConv2D(64, (3,3), strides=(1,1), data_format='channels_last', padding='same', depth_multiplier=1, activation='relu'))
      if BN == True:
          model.add(layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, \
                                              center=True, scale=True, beta_initializer='zeros', \
                                              gamma_initializer='ones', moving_mean_initializer='zeros', \
                                              moving_variance_initializer='ones', beta_regularizer=None, \
                                              gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
      #model.add(layers.Dropout(0.5))
      model.add(layers.ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
      model.add(layers.Dropout(0.5))
      model.add(layers.Conv2D(64, (1,1), strides=(1,1), padding='same', use_bias=False))
      if BN == True:
          model.add(layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, \
                                              center=True, scale=True, beta_initializer='zeros', \
                                              gamma_initializer='ones', moving_mean_initializer='zeros', \
                                              moving_variance_initializer='ones', beta_regularizer=None, \
                                              gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
      model.add(layers.Dropout(0.5))
      model.add(layers.ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
      model.add(layers.Dropout(0.5))

      # 5
      model.add(layers.SeparableConv2D(64, (3,3), strides=(1,1), data_format='channels_last', padding='same', depth_multiplier=1, activation='relu'))
      if BN == True:
          model.add(layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, \
                                              center=True, scale=True, beta_initializer='zeros', \
                                              gamma_initializer='ones', moving_mean_initializer='zeros', \
                                              moving_variance_initializer='ones', beta_regularizer=None, \
                                              gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
      #model.add(layers.Dropout(0.5))
      model.add(layers.ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
      model.add(layers.Dropout(0.5))
      model.add(layers.Conv2D(64, (1,1), strides=(1,1), padding='same', use_bias=False))
      if BN == True:      
          model.add(layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, \
                                              center=True, scale=True, beta_initializer='zeros', \
                                              gamma_initializer='ones', moving_mean_initializer='zeros', \
                                              moving_variance_initializer='ones', beta_regularizer=None, \
                                              gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
      #model.add(layers.Dropout(0.5))
      model.add(layers.ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
      model.add(layers.Dropout(0.5))


      
           
      
# Final layer
      
      #model.add(layers.Dropout(0.5))
      model.add(layers.AveragePooling2D(pool_size=(5, 25), strides=(2,2), padding='valid', data_format=None))
      model.add(layers.Dropout(0.5))
      model.add(layers.Flatten(data_format=None))
     # model.add(layers.Dropout(0.5))
      model.add(layers.Dense(64, activation='relu'))
      model.add(layers.Dropout(0.5))
      #model.add(layers.Dense(12, activation='softmax'))
      model.add(layers.Dense(5, activation='softmax'))
      #model.add(layers.Dropout(0.5))
      # Compilation step to choose loss function, optimizer and metric
      # Configuring the learning process
      model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
      #model.fit()
      history = model.fit(X_train, y_train_hot, batch_size=100, epochs=200, verbose=1, validation_data=(X_test, y_test_hot))
      
      #plt.plot(history.history['acc'])
      #Restarts layer sequence number 
      #K.clear_session()

      acc = history.history['acc']
      val_acc = history.history['val_acc']
      loss = history.history['loss']
      val_loss = history.history['val_loss']
      epochs = range(1, len(acc) + 1)
      
      plt.plot(epochs, acc, 'bo', label='Training acc')
      plt.plot(epochs, val_acc, 'b', label='Validation acc')
      plt.title('Training and validation accuracy')
      plt.legend()

      plt.figure()

      plt.plot(epochs, loss, 'bo', label='Training loss')
      plt.plot(epochs,val_loss, 'b', label='Validation loss')
      plt.title('Training and validation loss')
      plt.legend()
      plt.show()
      
      
      model.save('/content/target_model/speech_recognition.h5')
  
dscnn_train()  

In [0]:
mfcc_bed = wav2mfcc('test_single_file/bed/0135f3f2_nohash_0.wav', max_pad_len=51)
mfcc_cat = wav2mfcc('test_single_file/cat/069ab0d5_nohash_0.wav', max_pad_len=51)
mfcc_dog = wav2mfcc('test_single_file/dog/012c8314_nohash_0.wav', max_pad_len=51)
mfcc_learn = wav2mfcc('test_single_file/learn/234d6a48_nohash_1.wav', max_pad_len=51)
mfcc_sheila = wav2mfcc('test_single_file/sheila/00970ce1_nohash_0.wav', max_pad_len=51)


bed_file = mfcc_bed.reshape(1, 10, 51, 1)
cat_file = mfcc_cat.reshape(1, 10, 51, 1)
dog_file = mfcc_dog.reshape(1, 10, 51, 1)
learn_file = mfcc_learn.reshape(1, 10, 51, 1)
sheila_file = mfcc_sheila.reshape(1, 10, 51, 1)

In [149]:
#prediction = model.predict(np.array(tk.texts_to_sequences(text)))
prediction = model.predict(dog_file, batch_size=None, verbose=0, steps=None)
class_prediction = prediction.tolist()

for record in class_prediction: 
    #print(record)
    record

n = len(record)


def predict(record, n):

  
     
    max = record[0] 
    j = 0
 
    for i in range(1, n):
        
        if record[i] > max: 
            
            max = record[i]
            
        
    index = record.index(max)        
    return max, index  
  
    
predicted_class, indice = predict(record,n) 


print('Predicted word: ', labels[indice])



Predicted word:  dog
